# Dataset ingestion

This jupyter noteebook ingests the [Geocoded National Address File][gnaf] ([GNAF][gnaf]) from [data.gov.au](data.gov.au). It also downloads the [land values for NSW][nswlv], and ABS shapefiles 

It loads it all this data into a PostgreSQL database in a docker container, treating it like a disposable sqlite data store. It also downloads the ABS shape files as well as the 

Here we are going to ingest all the data necessary in order to assess land by land values, and filter them by address information. 

### The Steps

1. Download static assets and datasets
2. Setup a docker container with postgresql with GIS capabilities.
3. Ingest the [ABS shape files][abssf]
4. Ingest the latest [NSW valuer general land values][nswlv].
5. Ingest the [Geocoded National Address File][gnaf] ([GNAF][gnaf]) dataset
6. Link NSW Valuer General data with GNAF dataset

[gnaf]: https://data.gov.au/data/dataset/geocoded-national-address-file-g-naf
[nswlv]: https://www.valuergeneral.nsw.gov.au/land_value_summaries/lv.php
[abssf]: https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files

### Note

- Make sure docker is running first.

### Warning

Do not connect this to another database unless you've taken the time to update this, as it'll drop the existing database. I suggest instead take what you need from this script and disregard the rest. DO NOT USE DATABASE CREDENTIALS HERE FOR ANY OTHER STORE (especailly anything with drop permissions).

It also executes sql from a zip file downloaded from an external source.


## Configuration

These are some fields to configure if you wish to configure how the data is injected.

In [1]:
from lib.service.docker.defaults import INSTANCE_1_IMAGE_CONF, INSTANCE_1_CONTAINER_CONF
from lib.service.database.defaults import DB_INSTANCE_1_CONFIG

GLOBAL_FLAGS = {
    # If you mark this as true, the table `nsw_valuer_general.raw_entries`
    # will be dropped. If you have space limitations and no desire to debug
    # the data than dropping this makes sense. If you wish to debug some values
    # then keeping this around may make some sense.
    'drop_raw_nsw_valuer_general_entries': True,
    'reinitialise_container': True,
}

db_service_config = DB_INSTANCE_1_CONFIG
docker_image_conf = INSTANCE_1_IMAGE_CONF
docker_container_conf = INSTANCE_1_CONTAINER_CONF

## Download Static Files

Here we are downloading static files, as well as fetching the most recently published land values from the valuer generals website.

In [2]:
import logging
from lib.service.io import IoService
from lib.tasks.fetch_static_files import initialise, get_session

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

io_service = IoService.create(None)
async with get_session(io_service) as session:
    environment = await initialise(io_service, session)

land_value_dis = environment.land_value
w_sale_price = environment.sale_price_weekly
a_sale_price = environment.sale_price_annual
gnaf_dis = environment.gnaf

2024-10-02 21:07:37,391 - INFO - Checking Target "abs_main_structures.zip"
2024-10-02 21:07:37,392 - INFO - Checking Target "non_abs_shape.zip"
2024-10-02 21:07:37,392 - INFO - Checking Target "g-naf_aug24_allstates_gda2020_psv_1016.zip"
2024-10-02 21:07:37,392 - INFO - Checking Target "nswvg_lv_01_Oct_2024.zip"
2024-10-02 21:07:37,392 - INFO - Checking Target "nswvg_wps_01_Jan_2024.zip"
2024-10-02 21:07:37,393 - INFO - Checking Target "nswvg_wps_08_Jan_2024.zip"
2024-10-02 21:07:37,393 - INFO - Checking Target "nswvg_wps_15_Jan_2024.zip"
2024-10-02 21:07:37,393 - INFO - Checking Target "nswvg_wps_22_Jan_2024.zip"
2024-10-02 21:07:37,394 - INFO - Checking Target "nswvg_wps_29_Jan_2024.zip"
2024-10-02 21:07:37,394 - INFO - Checking Target "nswvg_wps_05_Feb_2024.zip"
2024-10-02 21:07:37,394 - INFO - Checking Target "nswvg_wps_12_Feb_2024.zip"
2024-10-02 21:07:37,395 - INFO - Checking Target "nswvg_wps_19_Feb_2024.zip"
2024-10-02 21:07:37,395 - INFO - Checking Target "nswvg_wps_26_Feb_202

## Create Container with Database

Here we are creating a container in docker from an image that uses the postgres image, which also installs a few extensions.

### Note

This notebook this is designed to be run more than once, so it'll throw away any existing container and database before creating a new one. After getting rid of any container using the same identifer, it'll create a new one and pull the relevant image if it's not already installed. It'll wait till the postgres instance is live then create the database. 

In [3]:
from lib.pipeline.gnaf.init_schema import init_target_schema
from lib.service.docker import DockerService
from lib.service.database import DatabaseService

docker_service = DockerService.create()

if GLOBAL_FLAGS['reinitialise_container']:
    image = docker_service.create_image(docker_image_conf)
    image.prepare()

    container = docker_service.create_container(image, docker_container_conf)
    container.clean()
    container.prepare(db_service_config)
    container.start()
else:
    print('skipping container initialisation')

db_service = DatabaseService.create(db_service_config, 32)
await db_service.wait_till_running()
await db_service.open()

if GLOBAL_FLAGS['reinitialise_container']:
    await init_target_schema(gnaf_dis.publication, io_service, db_service)
else:
    print('skipping DB initialisation')
    raise Exception()

2024-10-02 21:07:43,826 - INFO - running ./_out_zip/g-naf_aug24_allstates_gda2020_psv_1016/G-NAF/Extras/GNAF_TableCreation_Scripts/create_tables_ansi.sql
2024-10-02 21:07:43,846 - INFO - running ./_out_zip/g-naf_aug24_allstates_gda2020_psv_1016/G-NAF/Extras/GNAF_TableCreation_Scripts/add_fk_constraints.sql
2024-10-02 21:07:43,890 - INFO - running sql/move_gnaf_to_schema.sql


## Create Schema

This initialises the schema used by the different tables.

In [4]:
from lib.tasks.update_schema import update_schema, UpdateSchemaConfig

await update_schema(
    UpdateSchemaConfig(apply=True),
    db_service,
    io_service,
)

2024-10-02 21:07:43,901 - INFO - initalising nsw_vg db schema
2024-10-02 21:07:43,909 - INFO - running sql/meta/schema/001_APPLY_init.sql
2024-10-02 21:07:43,916 - INFO - running sql/abs/schema/001_APPLY_init.sql
2024-10-02 21:07:43,917 - INFO - running sql/nsw_lrs/schema/001_APPLY_init.sql
2024-10-02 21:07:43,920 - INFO - running sql/nsw_environment/schema/001_APPLY_init.sql
2024-10-02 21:07:43,924 - INFO - running sql/nsw_property/schema/001_APPLY_init.sql
2024-10-02 21:07:43,933 - INFO - running sql/nsw_vg/schema/001_APPLY_init.sql
2024-10-02 21:07:43,934 - INFO - running sql/nsw_vg/schema/002_APPLY_create_raw.sql
2024-10-02 21:07:43,944 - INFO - running sql/nsw_vg/schema/003_APPLY_shared_tables.sql
2024-10-02 21:07:43,957 - INFO - running sql/nsw_vg/schema/004_APPLY_land_value_tables.sql


## Consume the ABS Shapefiles

The [ABS provides a number of shape files][all abs shape files], we're going focus on 2 main sets of shapes. The **ABS Main Structures** which is stuff like SA1, 2, 3 & 4 along with greater cities, meshblocks, and states. As well as **Non ABS Main Structures** which is stuff like electoral divisions, suburbs post codes etc.

[all abs shape files]: https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files

### ABS Main Structures 

Any address or region we look up in the GNAF dataset, we want to visualise. The ABS has a few different geographic groups which we can visualise the data against, but each address in the GNAF dataset has a meshblock id, which is the smaller block the ABS breaks addresses up into for SA1, SA2, SA3 and SA4's.

This dataset is pretty useful for visualising the GNAF data for that reason.

### Non Abs Main Structures 

We are mostly ingesting these to make it simpler to narrow data of interest. Typically if you're looking at this data, you're probably doing it some scope of relevance, such as a local government area, an electorate division, or whatever.

In [5]:
from lib.tasks.ingest_abs import ingest_all

await ingest_all(db_service)

2024-10-02 21:07:47,379 - INFO - Populated abs_main_structures.state with 10/10 rows.
2024-10-02 21:07:50,256 - INFO - Populated abs_main_structures.gccsa with 35/35 rows.
2024-10-02 21:07:53,970 - INFO - Populated abs_main_structures.sa4 with 108/108 rows.
2024-10-02 21:07:58,494 - INFO - Populated abs_main_structures.sa3 with 359/359 rows.
2024-10-02 21:08:04,953 - INFO - Populated abs_main_structures.sa2 with 2473/2473 rows.
2024-10-02 21:08:20,255 - INFO - Populated abs_main_structures.sa1 with 61845/61845 rows.
2024-10-02 21:09:07,115 - INFO - Populated abs_main_structures.meshblock with 368286/368286 rows.
2024-10-02 21:09:20,437 - INFO - Populated non_abs_main_structures.localities with 15353/15353 rows.
2024-10-02 21:09:24,704 - INFO - Populated non_abs_main_structures.state_electoral_division_2021 with 452/452 rows.
2024-10-02 21:09:29,100 - INFO - Populated non_abs_main_structures.state_electoral_division_2022 with 452/452 rows.
2024-10-02 21:09:33,458 - INFO - Populated non_

## Ingesting NSW Land Values

First lets just get the CSV's into the database, then we'll break it up into seperates tables, then we'll form links with the GNAF dataset.

#### Documentation on this dataset

The valuer general website has a link to documentation on interpretting that data on [this page](https://www.nsw.gov.au/housing-and-construction/land-values-nsw/resource-library/land-value-information-user-guide). I didn't link to the PDF directly as it occasionally updated and a direct link is at risk of going stale.

It's useful getting the meaning behind the codes and terms used in the bulk data.

### Steps

1. **Build the `nsw_valuer_general.raw_entries_lv` table**: Here we are
   just loading the each file from the latest land value publication
   with minimal changes, and a bit of sanitisizing.
2. **Break CSV data into sepreate relations**, Just to break up the data
   into more efficent representations of the data, and data that will be
   easier to query, we're going to perform a series of queries against
   the GNAF data before using it populate the tables we care about.
3. **Parse contents of the property description**, The `property_description`
   from the original valuer general data constains alot of information. The
   most important of which is the land parcel or `lot/plan` information.
   There is other information in there as well.
4. If specified drop the raw entries consumed in step (tho the default is
   to do exactly that. 


In [6]:
from lib.tasks.nsw_vg.ingest_land_values import ingest_land_values, NswVgLandValueIngestionConfig

await ingest_land_values(
    NswVgLandValueIngestionConfig(
        keep_raw=not GLOBAL_FLAGS['drop_raw_nsw_valuer_general_entries'],
    ),
    db_service,
    environment.land_value.latest,
)

2024-10-02 21:10:17,531 - INFO - Step 1: Ingest raw files
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, encoding='utf-8')
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, encoding='utf-8')
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, encoding='utf-8')
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, enco

## Gnaf Ingestion

Here we ingest the GNAF dataset, this will take awhile.

In [7]:
from lib.tasks.ingest_gnaf import ingest_gnaf

async with db_service:
    await ingest_gnaf(gnaf_dis.publication, db_service)

2024-10-02 21:17:13,230 - INFO - Populating from Authority_Code_FLAT_TYPE_AUT_psv.psv
2024-10-02 21:17:13,235 - INFO - Populating from Authority_Code_ADDRESS_CHANGE_TYPE_AUT_psv.psv
2024-10-02 21:17:13,236 - INFO - Populating from Authority_Code_GEOCODE_TYPE_AUT_psv.psv
2024-10-02 21:17:13,236 - INFO - Populating from Authority_Code_MB_MATCH_CODE_AUT_psv.psv
2024-10-02 21:17:13,240 - INFO - Populating from Authority_Code_STREET_TYPE_AUT_psv.psv
2024-10-02 21:17:13,244 - INFO - Populating from Authority_Code_GEOCODED_LEVEL_TYPE_AUT_psv.psv
2024-10-02 21:17:13,247 - INFO - Populating from Authority_Code_LOCALITY_CLASS_AUT_psv.psv
2024-10-02 21:17:13,247 - INFO - Populating from Authority_Code_GEOCODE_RELIABILITY_AUT_psv.psv
2024-10-02 21:17:13,248 - INFO - Populating from Authority_Code_ADDRESS_ALIAS_TYPE_AUT_psv.psv
2024-10-02 21:17:13,249 - INFO - Populating from Authority_Code_LOCALITY_ALIAS_TYPE_AUT_psv.psv
2024-10-02 21:17:13,249 - INFO - Populating from Authority_Code_LEVEL_TYPE_AU

## Done

We've now built up the dataset, lets analysis what we got and show the contents of the database.

In [8]:
async with (
    db_service,
    await db_service.async_connect() as c,
    c.cursor() as cursor
):
    for schema in ['nsw_valuer_general', 'gnaf', 'abs_main_structures', 'non_abs_main_structures']:
        # Get the list of all tables
        cursor.execute(f"""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = '{schema}'
        """)
        tables = cursor.fetchall()
    
        # Get row count for each table
        for table in tables:
            await cursor.execute(f'SELECT COUNT(*) FROM {schema}.{table[0]}')
            count = cursor.fetchone()[0]
            print(f"Table {schema}.{table[0]} has {count} rows")

/var/folders/l1/1wl5vmds75qfv945_bs13x6w0000gn/T/ipykernel_12997/1949185990.py:1: RuntimeWarning: coroutine 'DatabaseService.async_connect' was never awaited
  async with db_service.async_connect() as c, c.cursor() as cursor:


TypeError: 'coroutine' object does not support the asynchronous context manager protocol